# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import pickle

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet'])

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('FigureEight', engine)
X = df.message.values
y = df[df.columns[4:]].values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip(),
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state = 0))))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
pipeline.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...nalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)
target_names = list(df.columns[4:])
for idx, target in enumerate(target_names):
    print(target)
    print(classification_report(y_test[:,idx], y_pred[:,idx]))

related
             precision    recall  f1-score   support

          0       0.65      0.55      0.59      1240
          1       0.86      0.91      0.88      3956
          2       0.83      0.25      0.38        40

avg / total       0.81      0.82      0.81      5236

request
             precision    recall  f1-score   support

          0       0.93      0.96      0.94      4330
          1       0.79      0.63      0.70       906

avg / total       0.90      0.91      0.90      5236

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5207
          1       0.00      0.00      0.00        29

avg / total       0.99      0.99      0.99      5236

aid_related
             precision    recall  f1-score   support

          0       0.79      0.82      0.81      3072
          1       0.73      0.69      0.71      2164

avg / total       0.77      0.77      0.77      5236

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {
    'tfidf__smooth_idf':[True, False],
    'clf__estimator__estimator__C': [1, 2, 5]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv = 5)
cv.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training example

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...nalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__smooth_idf': [True, False], 'clf__estimator__estimator__C': [1, 2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
y_pred = cv.predict(X_test)
target_names = list(df.columns[4:])
for idx, target in enumerate(target_names):
    print(target)
    print(classification_report(y_test[:,idx], y_pred[:,idx]))

related
             precision    recall  f1-score   support

          0       0.65      0.55      0.59      1240
          1       0.86      0.91      0.88      3956
          2       0.83      0.25      0.38        40

avg / total       0.81      0.82      0.81      5236

request
             precision    recall  f1-score   support

          0       0.93      0.96      0.94      4330
          1       0.79      0.63      0.70       906

avg / total       0.90      0.91      0.90      5236

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5207
          1       0.00      0.00      0.00        29

avg / total       0.99      0.99      0.99      5236

aid_related
             precision    recall  f1-score   support

          0       0.79      0.82      0.81      3072
          1       0.73      0.69      0.71      2164

avg / total       0.77      0.77      0.77      5236

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced', random_state=0)))
])
parameters = {
    'clf__estimator__max_depth': [3, 5, 10],
    'clf__estimator__n_estimators' : [10, 20]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv = 5)
cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...1, oob_score=False, random_state=0,
            verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [3, 5, 10], 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [10]:
y_pred = cv.predict(X_test)
target_names = list(df.columns[4:])
for idx, target in enumerate(target_names):
    print(target)
    print(classification_report(y_test[:,idx], y_pred[:,idx]))

related
             precision    recall  f1-score   support

          0       0.42      0.57      0.48      1240
          1       0.86      0.70      0.77      3956
          2       0.10      0.78      0.18        40

avg / total       0.75      0.67      0.70      5236

request
             precision    recall  f1-score   support

          0       0.93      0.86      0.89      4330
          1       0.51      0.71      0.59       906

avg / total       0.86      0.83      0.84      5236

offer
             precision    recall  f1-score   support

          0       0.99      0.99      0.99      5207
          1       0.05      0.10      0.06        29

avg / total       0.99      0.98      0.99      5236

aid_related
             precision    recall  f1-score   support

          0       0.74      0.75      0.75      3072
          1       0.64      0.62      0.63      2164

avg / total       0.70      0.70      0.70      5236

medical_help
             precision    recall  f1-sco

### 9. Export your model as a pickle file

In [11]:
pkl_filename = "classifier.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(cv.best_estimator_, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.